In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc matplotlib numpy

# CHSH-Ungleichung

*Nutzungsschätzung: Zwoi Minuta uff eme Heron r2-Prozessor (HINWEIS: Des isch nur e Schätzung. Dei Laufzeit ka variiere.)*
## Hintergrund
In dem Tutorial führet mr en Experiment uff eme Quantecomputer aus, um d'Verletzung vo dr CHSH-Ungleichung mit em Estimator-Primitiv z'zeige.

D'CHSH-Ungleichung, benannt nach de Autore Clauser, Horne, Shimony und Holt, wird verwendet, um Bells Theorem (1969) experimentell z'beweise. Des Theorem besagt, dass lokali Theorien mit verborgene Variabla manche Konsequenze vo dr Verschränkung in dr Quantemechanik ned erklärn könnet. D'Verletzung vo dr CHSH-Ungleichung wird verwendet, um z'zeige, dass d'Quantemechanik mit lokale Theorien mit verborgene Variabla ned vereinbar isch. Des isch en wichtigs Experiment zum Verschtändnis vo de Grundlage vo dr Quantemechanik.

Dr Nobelpreis für Physik 2022 isch an Alain Aspect, John Clauser und Anton Zeilinger verliehe worre, unter anderem für ihre Pionierarbeit in dr Quanteinformationswissenschaft und insbesondere für ihre Experimente mit verschränkte Photone, wo d'Verletzung vo de Bellsche Ungleichunge demonstriert hend.
## Anforderunge
Stellt vor em Beginn vo dem Tutorial sicher, dass Ihr Folgendes installiert hend:

* Qiskit SDK v1.0 oder neuer, mit [visualization](https://docs.quantum.ibm.com/api/qiskit/visualization)-Unterstützung
* Qiskit Runtime (`pip install qiskit-ibm-runtime`) v0.22 oder neuer
## Setup

In [1]:
# General
import numpy as np

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Qiskit Runtime imports
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator

# Plotting routines
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

## Schritt 1: Klassischi Eingabe uff en Quanteproblem abbilden
Für des Experiment erstelle mr en verschränkts Paar, wo mr jeds Qubit uff zwoi verschiedene Basa messe. Mir bezeichne d'Basa für s'erschte Qubit mit $A$ und $a$ und d'Basa für s'zweite Qubit mit $B$ und $b$. Des ermöglicht es uns, d'CHSH-Größe $S_1$ z'berechne:

$$
S_1 = A(B-b) + a(B+b).
$$

Jedi Observable isch entweder $+1$ oder $-1$. Klar isch, dass einer vo de Terme $B\pm b$ gleich $0$ sei muss und dr andere $\pm 2$ sei muss. Deshalb isch $S_1 = \pm 2$. Dr Durchschnittswert vo $S_1$ muss d'Ungleichung erfülle:

$$
|\langle S_1 \rangle|\leq 2.
$$

D'Entwicklung vo $S_1$ in Bezug uff $A$, $a$, $B$ und $b$ ergibt:

$$
|\langle S_1 \rangle| = |\langle AB \rangle - \langle Ab \rangle + \langle aB \rangle + \langle ab \rangle| \leq 2
$$

Ihr könnet e weiteri CHSH-Größe $S_2$ definiere:

$$
S_2 = A(B+b) - a(B-b),
$$

Des führt zu enr weitere Ungleichung:

$$
|\langle S_2 \rangle| = |\langle AB \rangle + \langle Ab \rangle - \langle aB \rangle + \langle ab \rangle| \leq 2
$$

Wenn d'Quantemechanik durch lokali Theorien mit verborgene Variabla beschriebe werre ka, müsset d'vorherige Ungleichunge wahr sei. Wie aber in dem Tutorial demonstriert wird, könnet dese Ungleichunge uff eme Quantecomputer verletzt werre. Deshalb isch d'Quantemechanik ned mit lokale Theorien mit verborgene Variabla vereinbar.
Falls Ihr mehr Theorie lerne wollet, erkundet [Entanglement in Action](/learning/courses/basics-of-quantum-information/entanglement-in-action/chsh-game) mit em John Watrous.
Ihr werdet en verschränkts Paar zwischen zwoi Qubits in eme Quantecomputer erstelle, indem Ihr de Bell-Zustand $|\Phi^+\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}}$ erzeuget. Mit em Estimator-Primitiv könnet Ihr direkt d'benötigte Erwartungswerte ($\langle AB \rangle, \langle Ab \rangle, \langle aB \rangle$ und $\langle ab \rangle$) kriege, um d'Erwartungswerte vo de beide CHSH-Größe $\langle S_1\rangle$ und $\langle S_2\rangle$ z'berechne. Vor dr Einführung vom Estimator-Primitiv hend mr d'Erwartungswerte aus de Messergebnisse konstruiere müsse.

Ihr werdet s'zweite Qubit in de $Z$- und $X$-Basa messe. S'erschte Qubit wird ebefalls in orthogonale Basa gmesse, aber mit eme Winkel bezüglich vom zweite Qubit, den mir zwischen $0$ und $2\pi$ variiere werdet. Wie Ihr sehet, macht s'Estimator-Primitiv s'Ausführe vo parametrisierte Schaltkreis sehr eifach. Anstatt e Reihe vo CHSH-Schaltkreise z'erstelle, müsset Ihr nur *oine* CHSH-Schaltkreis mit eme Parameter erstelle, wo de Messwinkel angibt, und e Reihe vo Phasewerte für de Parameter.

Schließlich werdet Ihr d'Ergebnisse analysiere und gege de Messwinkel auftrage. Ihr werdet sehe, dass für ene bestimmte Bereich vo Messwinkeln d'Erwartungswerte vo de CHSH-Größe $|\langle S_1\rangle| > 2$ oder $|\langle S_2\rangle| > 2$ send, was d'Verletzung vo dr CHSH-Ungleichung demonstriert.

In [2]:
# To run on hardware, select the backend with the fewest number of jobs in the queue
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
backend.name

'ibm_kingston'

### Create a parameterized CHSH circuit

First, we write the circuit with the parameter $\theta$, which we call `theta`. The [`Estimator` primitive](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) can enormously simplify circuit building and output analysis by directly providing expectation values of observables. Many problems of interest, especially for near-term applications on noisy systems, can be formulated in terms of expectation values. `Estimator` (V2) primitive can automatically change measurement basis based on the supplied observable.

In [3]:
theta = Parameter("$\\theta$")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)
chsh_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif" alt="Output of the previous code cell" />

### En parametrisierte CHSH-Schaltkreis erstelle
Zerscht schreibe mr de Schaltkreis mit em Parameter $\theta$, den mr `theta` nenne. S'[`Estimator`-Primitiv](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) ka de Schaltkreisaufbau und d'Ausgabeanalyse riesig vereinfache, indem s'direkt Erwartungswerte vo Observabla liefert. Viel interessanti Probleme, insbesondere für kurzfristige Anwendunge uff verrauschet Systeme, könnet in Form vo Erwartungswerte formuliert werre. S'`Estimator` (V2)-Primitiv ka automatisch d'Messbasis basierend uff dr bereitgestellte Observable ändere.

In [4]:
number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
# Phases need to be expressed as list of lists in order to work
individual_phases = [[ph] for ph in phases]

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif)

### E Lischt vo Phasewerte erstelle, wo später zugewiese werdet {#eine-liste-von-phasenwerten-erstellen-die-später-zugewiesen-werden}

Nachdem Ihr de parametrisierte CHSH-Schaltkreis erstellt hend, erstellt Ihr e Lischt vo Phasewerte, wo em Schaltkreis im nächste Schritt zugewiese werdet. Ihr könnet de folgende Code verwende, um e Lischt vo 21 Phasewerte im Bereich vo $0$ bis $2 \pi$ mit gleichem Abstand z'erstelle, also $0$, $0.1 \pi$, $0.2 \pi$, ..., $1.9 \pi$, $2 \pi$.

In [5]:
# <CHSH1> = <AB> - <Ab> + <aB> + <ab> -> <ZZ> - <ZX> + <XZ> + <XX>
observable1 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", -1), ("XZ", 1), ("XX", 1)]
)

# <CHSH2> = <AB> + <Ab> - <aB> + <ab> -> <ZZ> + <ZX> - <XZ> + <XX>
observable2 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", 1), ("XZ", -1), ("XX", 1)]
)

### Observabla
Jetzt brauche mr Observabla, aus dene mr d'Erwartungswerte berechne könnet. In unserm Fall betrachte mr orthogonali Basa für jeds Qubit, wobei d'parametrisierte $Y$-Rotation für s'erschte Qubit d'Messbasis nahezu kontinuierlich in Bezug uff d'Basis vom zweite Qubit variiert. Mr wähle deshalb d'Observabla $ZZ$, $ZX$, $XZ$ und $XX$.

In [6]:
target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

chsh_isa_circuit = pm.run(chsh_circuit)
chsh_isa_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif" alt="Output of the previous code cell" />

## Schritt 2: Problem für d'Ausführung uff Quantehardware optimiere {#schritt-2-problem-für-die-ausführung-auf-quantenhardware-optimieren}

Um d'Gesamtausführungszeit vom Job z'reduziere, akzeptiere V2-Primitive nur Schaltkreise und Observabla, wo de vom Zielsystem unterstützte Anweisunge und dr Konnektivität entsprechet (bezeichnet als Instruction Set Architecture (ISA)-Schaltkreise und -Observabla).

### ISA-Schaltkreis

In [7]:
isa_observable1 = observable1.apply_layout(layout=chsh_isa_circuit.layout)
isa_observable2 = observable2.apply_layout(layout=chsh_isa_circuit.layout)

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif)

### ISA-Observabla

Ebenso müsse mr d'Observabla transformiere, um se backend-kompatibel z'mache, bevor mr Jobs mit [`Runtime Estimator V2`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2#run) ausführe. Mr könnet d'Transformation mit dr `apply_layout`-Methode vom `SparsePauliOp`-Objekt durchführe.

In [8]:
# To run on a local simulator:
# Use the StatevectorEstimator from qiskit.primitives instead.

estimator = Estimator(mode=backend)

pub = (
    chsh_isa_circuit,  # ISA circuit
    [[isa_observable1], [isa_observable2]],  # ISA Observables
    individual_phases,  # Parameter values
)

job_result = estimator.run(pubs=[pub]).result()

## Schritt 3: Ausführe mit Qiskit-Primitive {#schritt-3-ausführen-mit-qiskit-primitiven}

Um s'gesamte Experiment in eim einzige Aufruf vom [`Estimator`](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) auszuführe.
Mr könnet en [Qiskit Runtime `Estimator`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2)-Primitiv erstelle, um unsere Erwartungswerte z'berechne. D'`EstimatorV2.run()`-Methode akzeptiert en Iterable vo `primitive unified blocs (PUBs)`. Jeds PUB isch en Iterable im Format `(circuit, observables, parameter_values: Optional, precision: Optional)`.

In [9]:
chsh1_est = job_result[0].data.evs[0]
chsh2_est = job_result[0].data.evs[1]

In [10]:
fig, ax = plt.subplots(figsize=(10, 6))

# results from hardware
ax.plot(phases / np.pi, chsh1_est, "o-", label="CHSH1", zorder=3)
ax.plot(phases / np.pi, chsh2_est, "o-", label="CHSH2", zorder=3)

# classical bound +-2
ax.axhline(y=2, color="0.9", linestyle="--")
ax.axhline(y=-2, color="0.9", linestyle="--")

# quantum bound, +-2√2
ax.axhline(y=np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.axhline(y=-np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.fill_between(phases / np.pi, 2, 2 * np.sqrt(2), color="0.6", alpha=0.7)
ax.fill_between(phases / np.pi, -2, -2 * np.sqrt(2), color="0.6", alpha=0.7)

# set x tick labels to the unit of pi
ax.xaxis.set_major_formatter(tck.FormatStrFormatter("%g $\\pi$"))
ax.xaxis.set_major_locator(tck.MultipleLocator(base=0.5))

# set labels, and legend
plt.xlabel("Theta")
plt.ylabel("CHSH witness")
plt.legend()
plt.show()

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/f6267448-0.avif" alt="Output of the previous code cell" />

In the figure, the lines and gray areas delimit the bounds; the outer-most (dash-dotted) lines delimit the quantum-bounds ($\pm 2$), whereas the inner (dashed) lines delimit the classical bounds ($\pm 2\sqrt{2}$). You can see that there are regions where the CHSH witness quantities exceeds the classical bounds. Congratulations! You have successfully demonstrated the violation of CHSH inequality in a real quantum system!

## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_3xxAgm1SF1wGp9k)